In [5]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/R6.csv')

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('spanish')

df.columns = ['ID', 'R6', "Conocimiento"]

texts = df['R6'].copy()
texts = [text.lower() for text in texts ]
texts = [text.split() for text in texts ]
texts = [[word.strip() for word in text] for text in texts]
texts = [[word for word in text if word not in stop_words ] for text in texts]

df["words"] = texts





from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_input(df, max_seq_len=None):
  texts = df['R6'].copy()
  labels = df['Conocimiento'].copy()

  stop_words = stopwords.words('spanish')

  texts = [text.lower() for text in texts ]
  texts = [text.split() for text in texts ]
  texts = [[word.strip() for word in text] for text in texts]
  texts = [[word for word in text if word not in stop_words ] for text in texts]

  vocab_length = 10000

  tokenizer = Tokenizer(num_words=vocab_length)
  tokenizer.fit_on_texts(texts)
  texts = tokenizer.texts_to_sequences(texts)

  if max_seq_len is None:
    max_seq_len = np.max([len(text) for text in texts])

  texts = pad_sequences(texts, maxlen=max_seq_len, padding='post')

  labels = np.array(labels)

  return texts, labels, max_seq_len, vocab_length, labels

texts, labels, max_seq_len, vocab_lenght, label_mapping = preprocess_input(df, 22)
print("Frases en el texto:\n", texts.shape)
print("\nEtiquetas:\n", labels.shape)
print("\nLongitud maxima de cada frase:\n", max_seq_len)
print("\nLongitud maxima de vocabulario:\n", vocab_lenght)
print("\nLabel mapping:\n", label_mapping)

import tensorflow as tf
import numpy as np
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras import regularizers

encoded_labels = np.zeros((len(labels), 3))  # Crear un array de ceros de forma (n, 3)

# Codificar los valores manualmente
for i, label in enumerate(labels):
    encoded_labels[i, label-1] = 1

labels = encoded_labels
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, train_size=0.7, random_state=220)
model = Sequential()
model.add(Embedding(input_dim=vocab_lenght, output_dim= 512, input_length=max_seq_len))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))  # Regularización mediante Dropout
model.add(LSTM(units=64))
model.add(Dense(units=32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(units=3, activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(texts_train, labels_train, epochs=5, batch_size=32)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(texts_test, labels_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Frases en el texto:
 (100, 22)

Etiquetas:
 (100,)

Longitud maxima de cada frase:
 22

Longitud maxima de vocabulario:
 10000

Label mapping:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
Epoch 1/5
3/3 [==============================] - 10s 156ms/step - loss: 1.5164 - accuracy: 0.4857
Epoch 2/5
3/3 [==============================] - 0s 145ms/step - loss: 1.4743 - accuracy: 0.7286
Epoch 3/5
3/3 [==============================] - 0s 151ms/step - loss: 1.4027 - accuracy: 0.7429
Epoch 4/5
3/3 [==============================] - 0s 142ms/step - loss: 1.3063 - accuracy: 0.7429
Epoch 5/5
1/1 [==============================] - 1s 962ms/step - loss: 1.1987 - accuracy: 0.6333
Test Loss: 1.1987252235412598
Test Accuracy: 0.6333333253860474
